In [3]:
import tensorflow as tf
import keras_tuner as kt

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full),(X_test,y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [5]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden",min_value = 0, max_value = 8, default = 2)
    n_neurons = hp.Int("n_neurons",min_value = 16, max_value = 256)
    learning_rate = hp.Float("learning_rate",min_value = 1e-4, max_value = 1e-2, sampling = "log")
    optimizer = hp.Choice("optimizer", values = ["sgd","adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons,activation = "relu"))
    model.add(tf.keras.layers.Dense(10,activation = "softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

In [6]:
random_search_tuner = kt.RandomSearch(
    build_model, objective = "val_accuracy", max_trials = 5, overwrite = True, directory = "my_fashion_mnist",
    project_name = "my_rnd_search", seed = 42)
random_search_tuner.search(X_train,y_train,epochs = 10, validation_data = (X_valid, y_valid))

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.8343999981880188

Best val_accuracy So Far: 0.8640000224113464
Total elapsed time: 00h 01m 45s


In [9]:
top3_models = random_search_tuner.get_best_models(num_models = 3)
best_model = top3_models[0]

In [10]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials= 3)
top3_params[0].values

{'n_hidden': 7,
 'n_neurons': 100,
 'learning_rate': 0.0012482904754698163,
 'optimizer': 'sgd'}

In [12]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials = 1)[0]
best_trial.summary()

Trial 1 summary
Hyperparameters:
n_hidden: 7
n_neurons: 100
learning_rate: 0.0012482904754698163
optimizer: sgd
Score: 0.8640000224113464


In [13]:
best_trial.metrics.get_last_value("val_accuracy")

0.8640000224113464

In [14]:
best_model.fit(X_train_full,y_train_full,epochs = 10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.8809 - loss: 0.3251
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 897us/step - accuracy: 0.8833 - loss: 0.3173
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 915us/step - accuracy: 0.8838 - loss: 0.3126
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 938us/step - accuracy: 0.8886 - loss: 0.2995
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8902 - loss: 0.2973
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8935 - loss: 0.2850
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8948 - loss: 0.2802
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8960 - loss: 0.2787
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9006 - loss: 0.2705
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8994 - loss: 0.2690
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - accuracy: 0.8599 - loss: 0.3995


In [15]:
print(test_loss, test_accuracy)

0.3931198716163635 0.8622999787330627
